In [1]:
%load_ext autoreload
%autoreload 2

!python --version

Python 3.7.6


In [2]:
import os

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [3]:
data = pd.read_csv('../data/fifa.csv', index_col=0)

In [4]:
from utils.transformers import MoneyTransformer, LengthTransformer, WeightTransformer

In [5]:
from utils.constants import SPECIFIC_POSITIONS


def transform_specific_position_value(value):
    if value is np.nan:
        return value
        
    return float(value.split('+')[0])
    
    
def transform_specific_position_grow(value):
    if value is np.nan:
        return value
        
    return float(value.split('+')[1])


class SpecificPositionTransformer(BaseEstimator, TransformerMixin):  
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        
        for specific_position in SPECIFIC_POSITIONS:
            X[specific_position + ' Value'] = X[specific_position].apply(transform_specific_position_value).astype('float64')
            X[specific_position + ' Grow'] = X[specific_position].apply(transform_specific_position_grow).astype('float64')

        return X

In [6]:
def transform_work_rate(index):
    def __transform_work_rate(value):
        if value is np.nan:
            return value
            
        return value.split('/')[index].strip()
        
    return __transform_work_rate
    

class WorkRateTransformer(BaseEstimator, TransformerMixin):  
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        
        X['Work Rate Attack'] = X['Work Rate'].apply(transform_work_rate(0)).astype('str')
        X['Work Rate Defense'] = X['Work Rate'].apply(transform_work_rate(1)).astype('str')

        return X

In [7]:
def transform_bool(value):
    return value.lower() == 'yes'


class BoolTransformer(BaseEstimator, TransformerMixin):  
    def __init__(self, col):
        self.col = col
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X[self.col] = X[self.col].astype('str').apply(transform_bool).astype('bool')
        return X

In [8]:
import re
import time

from datetime import datetime


months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def transform_date_time(value):
    if value is np.nan:
        return value
        
    if re.match(r"^[0-9]{4}$", value):
        date = datetime(int(value), 1, 1)
        return time.mktime(date.timetuple())
    
    match = re.search(r"^([a-zA-Z]+) ([0-9]+), ([0-9]{4})$", 'Jan 1, 2019')
    
    year = int(match[3])
    month = months.index(match[1]) + 1
    day = int(match[2])
        
    date = datetime(year, month, day)
    return time.mktime(date.timetuple())
    

class DateTransformer(BaseEstimator, TransformerMixin):  
    def __init__(self, col):
        self.col = col
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X[self.col] = X[self.col].astype('str').apply(transform_date_time).astype('int32')
        return X

In [9]:
class PositionTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        
        X['Position (4)'] = X['Position'].replace(to_replace=['ST', 'LW', 'RW', 'LS', 'RS', 'CF', 'RF', 'LF'], value= 'attack')
        X['Position (4)'] = X['Position (4)'].replace(to_replace=['CM', 'RM', 'LM', 'CAM', 'CDM', 'LCM', 'RCM', 'RDM', 'LDM', 'LAM', 'RAM'], value= 'middle')
        X['Position (4)'] = X['Position (4)'].replace(to_replace=['CB', 'LB', 'RB', 'RCB', 'LCB', 'RWB', 'LWB'], value= 'defense')
        X['Position (4)'] = X['Position (4)'].replace(to_replace=['GK'], value= 'goalkeeper')

        X['Position (13)'] = X['Position'].replace(to_replace=['ST', 'CF'], value= 'central attack')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['LS', 'LW', 'LF'], value= 'left attack')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['RS', 'RW', 'RF'], value= 'right attack')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['CAM', 'LAM', 'RAM'], value= 'attacking midfielder')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['RDM', 'LDM', 'CDM'], value= 'deffensive midfielder')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['CM', 'LCM', 'RCM'], value= 'midfielder')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['RM'], value= 'right midfielder')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['LM'], value= 'left midfielder')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['RWB', 'LWB'], value= 'middle')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['LB', 'LWB'], value= 'left defense')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['RB', 'RWB'], value= 'central defense')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['CB', 'RCB', 'LCB'], value= 'right defense')
        X['Position (13)'] = X['Position (13)'].replace(to_replace=['GK'], value= 'goalkeeper')

        return X

In [10]:
# https://github.com/timzatko/fiit-iau-project/blob/master/iau_predspracovanie_2.ipynb

pipeline = Pipeline([
    ('transform_wage', MoneyTransformer('Wage')),
    ('transform_value', MoneyTransformer('Value')),
    ('transform_release_clause', MoneyTransformer('Release Clause')),
    ('transform_height', LengthTransformer('Height')),
    ('transform_weight', WeightTransformer('Weight')),
    ('transform_specific_position', SpecificPositionTransformer()),
    ('transform_work_rate', WorkRateTransformer()),
    ('transform_real_face', BoolTransformer('Real Face')),
    ('transform_joined', DateTransformer('Joined')),
    ('transform_contract_valid_until', DateTransformer('Contract Valid Until')),
    ('transform_position', PositionTransformer()),
])

In [11]:
new_data = pipeline.transform(data.copy())

In [12]:
specific_positions_value = list(map(lambda x: x + ' Value', SPECIFIC_POSITIONS))
specific_positions_grow = list(map(lambda x: x + ' Grow', SPECIFIC_POSITIONS))

processed_attributes = ['Wage', 'Value', 'Height', 'Weight', 'Release Clause', 'Work Rate Attack', 'Work Rate Defense', 'Real Face', 'Joined', 'Contract Valid Until']

data_w_only_processed_attributes = new_data[processed_attributes + specific_positions_value + specific_positions_grow]

data_w_only_processed_attributes.head()

,Wage,Value,Height,Weight,Release Clause,Work Rate Attack,Work Rate Defense,Real Face,Joined,Contract Valid Until,...,CM Grow,RCM Grow,RM Grow,LWB Grow,LDM Grow,RDM Grow,LB Grow,LCB Grow,RCB Grow,RB Grow
ID,,,,,,,,,,,,,,,,,,,,,
158023,565000.0,110500000.0,176.76,159.0,226500000.0,Medium,Medium,True,1546300800,1609459200,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
20801,405000.0,77000000.0,189.84,183.0,127100000.0,High,Low,True,1546300800,1640995200,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
190871,290000.0,118500000.0,183.72,150.0,228100000.0,High,Medium,True,1546300800,1640995200,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
193080,260000.0,72000000.0,196.80,168.0,138600000.0,Medium,Medium,True,1546300800,1577836800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192985,355000.0,102000000.0,190.68,154.0,196400000.0,High,High,True,1546300800,1672531200,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [13]:
data_w_only_processed_attributes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18207 entries, 158023 to 246269
Data columns (total 52 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Wage                  17966 non-null  float64
 1   Value                 17955 non-null  float64
 2   Height                18159 non-null  float64
 3   Weight                18159 non-null  float64
 4   Release Clause        16643 non-null  float64
 5   Work Rate Attack      18207 non-null  object 
 6   Work Rate Defense     18207 non-null  object 
 7   Real Face             18207 non-null  bool   
 8   Joined                18207 non-null  int32  
 9   Contract Valid Until  18207 non-null  int32  
 10  LS Value              16122 non-null  float64
 11  ST Value              16122 non-null  float64
 12  RS Value              16122 non-null  float64
 13  LW Value              16122 non-null  float64
 14  LF Value              16122 non-null  float64
 15  CF Value     

In [14]:
new_data.to_csv('../data/fifa_processed.csv', index=False)